TO DO
- Develop code to handle no game / double game gameweeks (see, e.g. funtion get_was_home)

In [1]:
# number of latest gameweek
gameweek = 25
# check number of total players from fpl's site
total_players = 11250570

In [2]:
import pandas as pd
import numpy as np

import json
import requests
import sys

# allow more data columns to be shown than by default
pd.set_option('display.max_columns', 500)

In [3]:
sys.path.append('..')

In [4]:
import py_scripts.download_data as download_data

# Fetch FPL data

In [5]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

In [6]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
#fpl_data.set_index('id',inplace=True)
fpl_data['season'] = '2022-23'
fpl_data['name'] = fpl_data['first_name'] + ' ' + fpl_data['second_name']
# normalize spelling of names
fpl_data['name'] = fpl_data['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
#fpl_data['name'] = fpl_data['name'].map(player_name_dict).fillna(fpl_data['name'])
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
fpl_data['position'] = fpl_data['element_type'].map(position_dict)
# change team number to team name
fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]
fpl_data['GW'] = gameweek
fpl_data['value'] = fpl_data['now_cost']/10.0
fpl_data['selected'] = (fpl_data['selected_by_percent'].astype(float) / 100) * total_players
fpl_data['transfers_balance'] = fpl_data['transfers_in_event'] - fpl_data['transfers_out_event']
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance
0,NaN,NaN,84450,0,0,-1,1,1,3,3.3,5.6,6,Granit,2.3,3,False,,None,49,84450.jpg,3.7,Xhaka,2.7,False,None,a,Arsenal,3,93,843993,2257,731230,10159,0.5,19.0,Xhaka,2107,3,5,10,23,0,0,0,3,0,0,7,406,448.6,524.3,340.0,131.4,25,2.20,3.07,5.27,23.13,54,21,17,14,65,42,31,21,NaN,,3.0,,NaN,,0.09,0.0,0.13,0.23,0.99,0.98,232,132,153,66,78,32,113,38,1.07,0.43,2022-23,Granit Xhaka,MID,25,4.9,303765.39,-7902
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,0.0,0,Mohamed,0.0,4,False,Knee injury - Unknown return date,2023-01-26T10:00:07.778526Z,41,153256.jpg,1.2,Elneny,0.6,False,None,i,Arsenal,3,6,198169,276,269899,1551,0.0,1.5,Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00,0.04,0.04,1.29,479,219,431,212,555,224,482,223,NaN,,NaN,,NaN,,0.00,0.0,0.03,0.03,1.05,1.62,624,321,497,178,408,179,233,72,0.81,0.00,2022-23,Mohamed Elneny,MID,25,4.1,67503.42,-1275
2,NaN,NaN,156074,0,0,-3,3,0,2,1.2,1.4,0,Rob,0.2,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,Arsenal,3,7,7112,58,18308,112,0.0,1.7,Holding,17,0,0,0,0,0,0,0,0,0,0,0,23,3.4,0.1,0.0,0.3,0,0.00,0.00,0.00,0.28,485,172,490,177,561,222,503,177,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,1.48,0.00,595,169,348,123,440,162,435,162,0.00,0.00,2022-23,Rob Holding,DEF,25,4.2,11250.57,-54
3,100.0,75.0,167199,0,0,-3,3,0,3,2.0,2.2,2,Thomas,1.0,6,False,,2023-02-15T22:30:09.735511Z,47,167199.jpg,3.0,Partey,0.4,False,None,a,Arsenal,3,59,99934,432,131732,780,0.2,12.6,Partey,1591,2,0,9,11,0,0,0,2,0,0,5,307,353.4,310.8,174.0,84.0,18,1.63,1.36,3.00,18.18,95,40,62,50,145,89,92,64,NaN,,NaN,,NaN,,0.09,0.0,0.08,0.17,1.03,0.62,315,191,266,121,157,68,278,91,1.02,0.51,2022-23,Thomas Partey,MID,25,4.7,45002.28,-348
4,100.0,100.0,184029,0,0,5,-5,4,3,5.2,9.4,11,Martin,4.2,7,True,,2022-09-18T20:00:06.315087Z,70,184029.jpg,5.9,Ødegaard,38.3,False,None,a,Arsenal,3,142,5377123,78836,2823078,108051,0.6,20.3,Ødegaard,2041,9,7,10,22,0,0,0,3,0,0,21,542,619.6,810.4,632.0,206.4,24,6.57,5.56,12.17,23.29,14,5,5,4,19,11,8,6,3.0,,1.0,,NaN,,0.29,0.0,0.25,0.54,1.03,0.97,40,20,43,15,8,3,5,3,1.06,0.44,2022-23,Martin degaard,MID,25,7.0,4308968.31,-29215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,75.0,NaN,430871,0,0,0,0,0,4,0.8,1.5,1,Matheus,1.0,682,False,Ankle injury - 75% chance of playing,2023-02-24T23:30:08.241906Z,55,430871.jpg,1.3,Santos Car

In [7]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home_team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away_team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
fpl_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
0,2293083,NaN,False,False,274,None,0,False,None,14,NaN,5,NaN,[],3,3,75184,Brighton,Manchester Utd
1,2293085,NaN,False,False,276,None,0,False,None,9,NaN,12,NaN,[],2,4,75186,Liverpool,Fulham
2,2293086,NaN,False,False,277,None,0,False,None,19,NaN,13,NaN,[],2,5,75187,Manchester City,West Ham
3,2293057,NaN,False,False,248,None,0,False,None,5,NaN,15,NaN,[],3,4,75158,Newcastle Utd,Brighton
4,2292810,1.0,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74911,Crystal Palace,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2293185,38.0,False,False,376,2023-05-28T15:30:00Z,0,False,False,3,NaN,8,NaN,[],2,2,75286,Everton,Bournemouth
376,2293186,38.0,False,False,377,2023-05-28T15:30:00Z,0,False,False,18,NaN,11,NaN,[],3,2,75287,Leeds United,Tottenham
377,2293187,38.0,False,False,378,2023-05-28T15:30:00Z,0,False,False,19,NaN,10,NaN,[],2,3,75288,Leicester City,West Ham
378,2293188,38.0,False,False,379,2023-05-28T15:30:00Z,0,False,False,9,NaN,14,NaN,[],2,4,75289,Manchester Utd,Fulham


In [8]:
def get_opponent(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    # my_team is the home team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team), \
                                         'away_team']
    # my_team is the away team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team), \
                                         'home_team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) )].shape[0]!=1:
        player = my_row['name']
        print(f'Number of games not equal to one for {player} ({my_team})')
        return np.nan

    return opponent_team.values[0]

In [9]:
def get_kickoff_time(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    kickoff_time = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) ), 'kickoff_time']
    if kickoff_time.shape[0]==1:
        return kickoff_time.values[0]
    else:
        player = my_row['name']
        print(f'Kickoff_time shape not one for {player} ({my_team})')
        return np.nan

In [10]:
def get_was_home(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        return 1
    elif fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        return 0
    else:
        player = my_row['name']
        print(f'Error in get_was_home for {player} ({my_team})')
        return np.nan

In [11]:
fpl_data['opp_team_name'] = fpl_data.apply(lambda x: get_opponent(x, fpl_fixtures, gameweek), axis=1)
fpl_data['kickoff_time'] = fpl_data.apply(lambda x: get_kickoff_time(x, fpl_fixtures, gameweek), axis=1)
fpl_data['was_home'] = fpl_data.apply(lambda x: get_was_home(x, fpl_fixtures, gameweek), axis=1)
fpl_data

Number of games not equal to one for Granit Xhaka (Arsenal)
Number of games not equal to one for Mohamed Elneny (Arsenal)
Number of games not equal to one for Rob Holding (Arsenal)
Number of games not equal to one for Thomas Partey (Arsenal)
Number of games not equal to one for Martin degaard (Arsenal)
Number of games not equal to one for Kieran Tierney (Arsenal)
Number of games not equal to one for Nicolas Pepe (Arsenal)
Number of games not equal to one for Benjamin White (Arsenal)
Number of games not equal to one for Eddie Nketiah (Arsenal)
Number of games not equal to one for Emile Smith Rowe (Arsenal)
Number of games not equal to one for Bukayo Saka (Arsenal)
Number of games not equal to one for Takehiro Tomiyasu (Arsenal)
Number of games not equal to one for Aaron Ramsdale (Arsenal)
Number of games not equal to one for Gabriel dos Santos Magalhaes (Arsenal)
Number of games not equal to one for Nuno Varela Tavares (Arsenal)
Number of games not equal to one for Gabriel Martinelli Si

Number of games not equal to one for James Milner (Liverpool)
Number of games not equal to one for Jordan Henderson (Liverpool)
Number of games not equal to one for Joel Matip (Liverpool)
Number of games not equal to one for Thiago Alcantara do Nascimento (Liverpool)
Number of games not equal to one for Alex Oxlade-Chamberlain (Liverpool)
Number of games not equal to one for Roberto Firmino (Liverpool)
Number of games not equal to one for Virgil van Dijk (Liverpool)
Number of games not equal to one for Alisson Ramses Becker (Liverpool)
Number of games not equal to one for Fabio Henrique Tavares (Liverpool)
Number of games not equal to one for Mohamed Salah (Liverpool)
Number of games not equal to one for Andrew Robertson (Liverpool)
Number of games not equal to one for Trent Alexander-Arnold (Liverpool)
Number of games not equal to one for Joseph Gomez (Liverpool)
Number of games not equal to one for Naby Keita (Liverpool)
Number of games not equal to one for Diogo Teixeira da Silva (L

Error in get_was_home for Pontus Jansson (Brentford)
Error in get_was_home for Christian Nrgaard (Brentford)
Error in get_was_home for Ivan Toney (Brentford)
Error in get_was_home for David Raya Martin (Brentford)
Error in get_was_home for Sergi Canos Tenes (Brentford)
Error in get_was_home for Josh Dasilva (Brentford)
Error in get_was_home for Kristoffer Ajer (Brentford)
Error in get_was_home for Rico Henry (Brentford)
Error in get_was_home for Vitaly Janelt (Brentford)
Error in get_was_home for Saman Ghoddos (Brentford)
Error in get_was_home for Mathias Jensen (Brentford)
Error in get_was_home for Yoane Wissa (Brentford)
Error in get_was_home for Mads Roerslev Rasmussen (Brentford)
Error in get_was_home for Mads Bech Srensen (Brentford)
Error in get_was_home for Ethan Pinnock (Brentford)
Error in get_was_home for Frank Onyeka (Brentford)
Error in get_was_home for Shandon Baptiste (Brentford)
Error in get_was_home for Bryan Mbeumo (Brentford)
Error in get_was_home for Charlie Goode (B

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
0,NaN,NaN,84450,0,0,-1,1,1,3,3.3,5.6,6,Granit,2.3,3,False,,None,49,84450.jpg,3.7,Xhaka,2.7,False,None,a,Arsenal,3,93,843993,2257,731230,10159,0.5,19.0,Xhaka,2107,3,5,10,23,0,0,0,3,0,0,7,406,448.6,524.3,340.0,131.4,25,2.20,3.07,5.27,23.13,54,21,17,14,65,42,31,21,NaN,,3.0,,NaN,,0.09,0.0,0.13,0.23,0.99,0.98,232,132,153,66,78,32,113,38,1.07,0.43,2022-23,Granit Xhaka,MID,25,4.9,303765.39,-7902,NaN,NaN,1.0
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,0.0,0,Mohamed,0.0,4,False,Knee injury - Unknown return date,2023-01-26T10:00:07.778526Z,41,153256.jpg,1.2,Elneny,0.6,False,None,i,Arsenal,3,6,198169,276,269899,1551,0.0,1.5,Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00,0.04,0.04,1.29,479,219,431,212,555,224,482,223,NaN,,NaN,,NaN,,0.00,0.0,0.03,0.03,1.05,1.62,624,321,497,178,408,179,233,72,0.81,0.00,2022-23,Mohamed Elneny,MID,25,4.1,67503.42,-1275,NaN,NaN,1.0
2,NaN,NaN,156074,0,0,-3,3,0,2,1.2,1.4,0,Rob,0.2,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,Arsenal,3,7,7112,58,18308,112,0.0,1.7,Holding,17,0,0,0,0,0,0,0,0,0,0,0,23,3.4,0.1,0.0,0.3,0,0.00,0.00,0.00,0.28,485,172,490,177,561,222,503,177,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,1.48,0.00,595,169,348,123,440,162,435,162,0.00,0.00,2022-23,Rob Holding,DEF,25,4.2,11250.57,-54,NaN,NaN,1.0
3,100.0,75.0,167199,0,0,-3,3,0,3,2.0,2.2,2,Thomas,1.0,6,False,,2023-02-15T22:30:09.735511Z,47,167199.jpg,3.0,Partey,0.4,False,None,a,Arsenal,3,59,99934,432,131732,780,0.2,12.6,Partey,1591,2,0,9,11,0,0,0,2,0,0,5,307,353.4,310.8,174.0,84.0,18,1.63,1.36,3.00,18.18,95,40,62,50,145,89,92,64,NaN,,NaN,,NaN,,0.09,0.0,0.08,0.17,1.03,0.62,315,191,266,121,157,68,278,91,1.02,0.51,2022-23,Thomas Partey,MID,25,4.7,45002.28,-348,NaN,NaN,1.0
4,100.0,100.0,184029,0,0,5,-5,4,3,5.2,9.4,11,Martin,4.2,7,True,,2022-09-18T20:00:06.315087Z,70,184029.jpg,5.9,Ødegaard,38.3,False,None,a,Arsenal,3,142,5377123,78836,2823078,108051,0.6,20.3,Ødegaard,2041,9,7,10,22,0,0,0,3,0,0,21,542,619.6,810.4,632.0,206.4,24,6.57,5.56,12.17,23.29,14,5,5,4,19,11,8,6,3.0,,1.0,,NaN,,0.29,0.0,0.25,0.54,1.03,0.97,40,20,43,15,8,3,5,3,1.06,0.44,2022-23,Martin degaard,MID,25,7.0,4308968.31,-29215,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,75.0,NaN,430871,0,0,0,0,0,4,0.8,1.5,1,Matheus

In [12]:
fpl_data.to_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv')
fpl_fixtures.to_csv('../data/fpl_fixtures.csv')

In [13]:
difference_columns = [
    'total_points', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
    'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed',
    'penalties_saved', 'saves', 'threat', 'yellow_cards', 'red_cards'
]

In [14]:
def calculate_difference_to_previous_gameweek(my_row):
    my_id = my_row['id']
    if my_id in old_ids:
        for my_column in difference_columns:
            my_row[my_column] = (my_row[my_column] -
                                     fpl_previous.loc[fpl_previous['id']==my_id, my_column]).values[0]
        return my_row
    elif my_id in new_ids:
        # for new players, these should be the same already, but just in case
        my_row['total_points'] = my_row['event_points']
        return my_row
    else:
        print(f'Player id neither old or new: {my_id}')
        return np.nan

In [15]:
# create a csv that contains each week's fpl data in one table
if gameweek==1:
    fpl_data['total_points'] = fpl_data['event_points']
    fpl_data[fpl_data['minutes']>0].to_csv('../data/fpl_data.csv')
    print(f'Gameweek {gameweek} data saved.')
elif gameweek>1:
    fpl_previous = pd.read_csv(f'../data/fpl_data/fpl_week_{gameweek-1}.csv')
    new_ids = set(fpl_data['id']).difference(set(fpl_previous['id']))
    old_ids = set(fpl_data['id']).intersection(set(fpl_previous['id']))
    # convert all difference columns to floats
    fpl_data[difference_columns] = fpl_data[difference_columns].astype(float)
    fpl_gameweek = fpl_data.apply(lambda x: calculate_difference_to_previous_gameweek(x), axis=1)
    fpl_gameweek = fpl_gameweek[fpl_gameweek['minutes']>0]
    print(f'Number of players that played this week: {fpl_gameweek.shape[0]}')
    fpl_previous_weeks = pd.read_csv('../data/fpl_data.csv', index_col=0)
    print(f'Size of fpl_previous_weeks: {fpl_previous_weeks.shape[0]}')
    #fpl_previous_weeks = fpl_previous_weeks[fpl_previous_weeks['GW']<gameweek]
    fpl_all_weeks = pd.concat([fpl_previous_weeks, fpl_gameweek], axis=0)
    print(f'Size of fpl_all_weeks: {fpl_all_weeks.shape[0]}')
    display(fpl_all_weeks)
else:
    print('Check gameweek!')

Number of players that played this week: 245
Size of fpl_previous_weeks: 6753
Size of fpl_all_weeks: 6998


,id,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,starts_per_90,clean_sheets_per_90
0,3,NaN,NaN,84450,0,0,0,0,0,3,2.1,2.1,2,Granit,2.0,False,NaN,NaN,50,84450.jpg,2.0,Xhaka,0.7,False,NaN,a,Arsenal,3,2.0,8050,8050,8788,8788,0.4,0.4,Xhaka,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,12.0,16.6,15.0,2.0,3.4,98,32,81,52,167,90,123,65,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Granit Xhaka,MID,1,5.0,60582.963,-738,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,167199,0,0,0,0,0,3,2.1,2.1,3,Thomas,3.0,False,NaN,NaN,50,167199.jpg,3.0,Partey,0.7,False,NaN,a,Arsenal,3,3.0,11320,11320,7903,7903,0.6,0.6,Partey,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,22.6,12.9,10.0,4.6,61,23,95,58,96,57,85,48,NaN,NaN,4.0,NaN,NaN,NaN,2022-23,Thomas Partey,MID,1,5.0,60582.963,3417,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,184029,0,0,0,0,0,3,2.8,2.8,3,Martin,3.0,False,NaN,NaN,65,184029.jpg,3.0,Ødegaard,13.4,False,NaN,a,Arsenal,3,3.0,52695,52695,121593,121593,0.5,0.5,Ødegaard,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,3.4,8.3,23.0,3.5,220,96,116,68,52,29,119,64,3.0,NaN,1.0,NaN,NaN,NaN,2022-23,Martin degaard,MID,1,6.5,1159731.006,-68898,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,100.0,75.0,192895,0,0,0,0,0,2,2.8,2.1,1,Kieran,1.0,False,NaN,2022-08-04T15:00:05.276574Z,50,192895.jpg,1.0,Tierney,1.2,False,NaN,a,Arsenal,3,1.0,2682,2682,16585,16585,0.2,0.2,Tierney,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,0.2,0.0,0.6,195,85,249,88,491,201,245,88,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Kieran Tierney,DEF,1,5.0,103856.508,-13903,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,NaN,NaN,198869,0,0,0,0,0,2,2.4,2.4,5,Benjamin,5.0,False,NaN,NaN,45,198869.jpg,5.0,White,6.4,False,NaN,a,Arsenal,3,5.0,30058,30058,35607,35607,1.1,1.1,White,90.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25.0,31.4,12.0,0.0,4.3,36,10,99,32,349,150,90,24,NaN,NaN,NaN,NaN,NaN,NaN,2022-23,Ben White,DEF,1,4.5,553901.376,-5549,Crystal Palace,2022-08-05T19:00:00Z,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [16]:
# save data
if gameweek>1:
    fpl_all_weeks.to_csv('../data/fpl_data.csv')

# Fetch fbref data

In [17]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
links = download_data.get_match_report_links(url)
links

['https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 'https://fbref.com/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/1ac96eb4/Newcastle-United-Nottingham-Forest-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/82702941/Leeds-United-Wolverhampton-Wanderers-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/877e3193/Bournemouth-Aston-Villa-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3a917cee/Everton-Chelsea-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3249ba27/Leicester-City-Brentford-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/8251694e/Manchester-United-Brighton-and-Hove-Albion-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/ece62baf/West-Ham-United-Manchester-City-August-7-2022-Premier-League',
 'https:

In [18]:
# check which matches (links) are already collected
with open('../data/fbref_fixture_links.txt') as f:
    old_links = f.read().splitlines()
old_links

['https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 'https://fbref.com/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/1ac96eb4/Newcastle-United-Nottingham-Forest-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/82702941/Leeds-United-Wolverhampton-Wanderers-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/877e3193/Bournemouth-Aston-Villa-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3a917cee/Everton-Chelsea-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3249ba27/Leicester-City-Brentford-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/8251694e/Manchester-United-Brighton-and-Hove-Albion-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/ece62baf/West-Ham-United-Manchester-City-August-7-2022-Premier-League',
 'https:

In [19]:
# get links to games for which data has not been collected yet
new_links = [link for link in links if link not in old_links]
new_links

['https://fbref.com/en/matches/b452508e/Fulham-Wolverhampton-Wanderers-February-24-2023-Premier-League',
 'https://fbref.com/en/matches/145cc0a5/Leicester-City-Arsenal-February-25-2023-Premier-League',
 'https://fbref.com/en/matches/25436a3b/Everton-Aston-Villa-February-25-2023-Premier-League',
 'https://fbref.com/en/matches/767981c6/West-Ham-United-Nottingham-Forest-February-25-2023-Premier-League',
 'https://fbref.com/en/matches/9edb32a3/Leeds-United-Southampton-February-25-2023-Premier-League',
 'https://fbref.com/en/matches/e731c1dd/Bournemouth-Manchester-City-February-25-2023-Premier-League',
 'https://fbref.com/en/matches/8a59c49c/Crystal-Palace-Liverpool-February-25-2023-Premier-League',
 'https://fbref.com/en/matches/8eff71b5/Tottenham-Hotspur-Chelsea-February-26-2023-Premier-League',
 'https://fbref.com/en/matches/705a2f3c/Arsenal-Everton-March-1-2023-Premier-League',
 'https://fbref.com/en/matches/5e05f5f4/Liverpool-Wolverhampton-Wanderers-March-1-2023-Premier-League']

In [20]:
fbref_data = download_data.download_match_report_data(new_links)
# normalize spelling of player names
fbref_data[('General','Player')] = fbref_data[('General','Player')].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
# map certain player names such that they are the same for both fpl and fbref data
#fbref_data[('General','Player')] = fbref_data[('General','Player')].map(player_name_dict).fillna(fbref_data[('General','Player')])
fbref_data

100%|███████████████████████████████████████████| 10/10 [00:47<00:00,  4.70s/it]


General                                Performance               \
              Player     #   Nation Pos     Age Min         Gls Ast PK PKatt   
0    Carlos Vinicius  30.0   br BRA  FW  27-336  90           0   0  0     0   
1            Willian  20.0   br BRA  LW  34-199  82           0   0  0     0   
2       Harry Wilson   8.0  wls WAL  LW  25-339   8           0   0  0     0   
3         Bobby Reid  14.0   jm JAM  RW  30-022  45           0   0  0     0   
4      Manor Solomon  11.0   il ISR  RW  23-215  45           1   0  0     0   
..               ...   ...      ...  ..     ...  ..         ...  .. ..   ...   
291  Rayan Ait Nouri   3.0   fr FRA  LB  21-268  67           0   0  0     0   
292       Max Kilman  23.0  eng ENG  CB  25-282  90           0   0  0     0   
293     Craig Dawson  15.0  eng ENG  CB  32-299  90           0   0  0     0   
294    Nelson Semedo  22.0   pt POR  RB  29-105  90           0   0  0     0   
295          Jose Sa   1.0   pt POR  GK  30-043  90           0   0  0     0   

                                            Expected           SCA     Passes  \
    Sh SoT CrdY CrdR Touches Tkl Int Blocks       xG npxG  xAG SCA GCA    Cmp   
0    2   2    0    0      30   0   0      0      0.2  0.2  0.0   0   0      8   
1    0   0    0    0      32   1   1      0      0.0  0.0  0.0   4   0     13   
2    0   0    0    0       6   0   1      0      0.0  0.0  0.0   1   0      3   
3    0   0    0    0      25   1   1      0      0.0  0.0  0.1   3   0     15   
4    1   1    0    0      25   1   1      0      0.0  0.0  0.0   1   0     14   
..  ..  ..  ...  ...     ...  ..  ..    ...      ...  ...  ...  ..  ..    ...   
291  0   0    0    0      54   3   2      1      0.0  0.0  0.0   0   0     31   
292  0   0    0    0      46   0   0      1      0.0  0.0  0.0   0   0     35   
293  0   0    0    0      58   1   1      1      0.0  0.0  0.0   0   0     39   
294  0   0    1    0      58   1   1      3      0.0  0.0  0.0   0   0     42   
295  0   0    0    0      46   0   0      0      0.0  0.0  0.0   0   0     30   

                   Carries      Take-Ons      General                      \
    Att  Cmp% PrgP Carries PrgC      Att Succ   Squad HomeTeam   Opponent   
0    19  42.1    1      17    0        0    0  Fulham     True     Wolves   
1    21  61.9    0      22    2        4    1  Fulham     True     Wolves   
2     5  60.0    1       7    0        1    1  Fulham     True     Wolves   
3    21  71.4    0      14    2        2    2  Fulham     True     Wolves   
4    19  73.7    2      18    3        2    1  Fulham     True     Wolves   
..   ..   ...  ...     ...  ...      ...  ...     ...      ...        ...   
291  41  75.6    4      32    3        3    3  Wolves    False  Liverpool   
292  41  85.4    1      26    0        1    1  Wolves    False  Liverpool   
293  47  83.0    2      35    1        1    1  Wolves    False  Liverpool   
294  49  85.7    4      38    3        2    2  Wolves    False  Liverpool   
295  43  69.8    0      29    0        0    0  Wolves    False  Liverpool   

    Total                           Short            Medium            Long  \
      Cmp Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%    Cmp Att   Cmp%  Cmp   
0       8  19  42.1      95      26     4  10   40.0      2   3   66.7    0   
1      13  21  61.9     217       9     7  11   63.6      4   5   80.0    2   
2       3   5  60.0      51       5     2   2  100.0      1   1  100.0    0   
3      15  21  71.4     230      86     9  10   90.0      5   7   71.4    1   
4      14  19  73.7     242      40     5   7   71.4      9  11   81.8    0   
..    ...  ..   ...     ...     ...   ...  ..    ...    ...  ..    ...  ...   
291    31  41  75.6     358     120    21  26   80.8      8  11   72.7    0   
292    35  41  85.4     693     281    12  12  100.0     19  19  100.0    4   
293    39  47  83.0     735     239    13  16   81.3     22  24   91.7    4   
294    42  49  85.7     600     282    27  28  

In [21]:
# change column name to align the new Opta data with the old Statsbomb data
fbref_data.columns = fbref_data.columns.values
fbref_data.columns = pd.MultiIndex.from_tuples(fbref_data.rename(columns={('Expected', 'xAG'): ('Expected', 'xA')}))
fbref_data

General                                Performance               \
              Player     #   Nation Pos     Age Min         Gls Ast PK PKatt   
0    Carlos Vinicius  30.0   br BRA  FW  27-336  90           0   0  0     0   
1            Willian  20.0   br BRA  LW  34-199  82           0   0  0     0   
2       Harry Wilson   8.0  wls WAL  LW  25-339   8           0   0  0     0   
3         Bobby Reid  14.0   jm JAM  RW  30-022  45           0   0  0     0   
4      Manor Solomon  11.0   il ISR  RW  23-215  45           1   0  0     0   
..               ...   ...      ...  ..     ...  ..         ...  .. ..   ...   
291  Rayan Ait Nouri   3.0   fr FRA  LB  21-268  67           0   0  0     0   
292       Max Kilman  23.0  eng ENG  CB  25-282  90           0   0  0     0   
293     Craig Dawson  15.0  eng ENG  CB  32-299  90           0   0  0     0   
294    Nelson Semedo  22.0   pt POR  RB  29-105  90           0   0  0     0   
295          Jose Sa   1.0   pt POR  GK  30-043  90           0   0  0     0   

                                            Expected           SCA     Passes  \
    Sh SoT CrdY CrdR Touches Tkl Int Blocks       xG npxG   xA SCA GCA    Cmp   
0    2   2    0    0      30   0   0      0      0.2  0.2  0.0   0   0      8   
1    0   0    0    0      32   1   1      0      0.0  0.0  0.0   4   0     13   
2    0   0    0    0       6   0   1      0      0.0  0.0  0.0   1   0      3   
3    0   0    0    0      25   1   1      0      0.0  0.0  0.1   3   0     15   
4    1   1    0    0      25   1   1      0      0.0  0.0  0.0   1   0     14   
..  ..  ..  ...  ...     ...  ..  ..    ...      ...  ...  ...  ..  ..    ...   
291  0   0    0    0      54   3   2      1      0.0  0.0  0.0   0   0     31   
292  0   0    0    0      46   0   0      1      0.0  0.0  0.0   0   0     35   
293  0   0    0    0      58   1   1      1      0.0  0.0  0.0   0   0     39   
294  0   0    1    0      58   1   1      3      0.0  0.0  0.0   0   0     42   
295  0   0    0    0      46   0   0      0      0.0  0.0  0.0   0   0     30   

                   Carries      Take-Ons      General                      \
    Att  Cmp% PrgP Carries PrgC      Att Succ   Squad HomeTeam   Opponent   
0    19  42.1    1      17    0        0    0  Fulham     True     Wolves   
1    21  61.9    0      22    2        4    1  Fulham     True     Wolves   
2     5  60.0    1       7    0        1    1  Fulham     True     Wolves   
3    21  71.4    0      14    2        2    2  Fulham     True     Wolves   
4    19  73.7    2      18    3        2    1  Fulham     True     Wolves   
..   ..   ...  ...     ...  ...      ...  ...     ...      ...        ...   
291  41  75.6    4      32    3        3    3  Wolves    False  Liverpool   
292  41  85.4    1      26    0        1    1  Wolves    False  Liverpool   
293  47  83.0    2      35    1        1    1  Wolves    False  Liverpool   
294  49  85.7    4      38    3        2    2  Wolves    False  Liverpool   
295  43  69.8    0      29    0        0    0  Wolves    False  Liverpool   

    Total                           Short            Medium            Long  \
      Cmp Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%    Cmp Att   Cmp%  Cmp   
0       8  19  42.1      95      26     4  10   40.0      2   3   66.7    0   
1      13  21  61.9     217       9     7  11   63.6      4   5   80.0    2   
2       3   5  60.0      51       5     2   2  100.0      1   1  100.0    0   
3      15  21  71.4     230      86     9  10   90.0      5   7   71.4    1   
4      14  19  73.7     242      40     5   7   71.4      9  11   81.8    0   
..    ...  ..   ...     ...     ...   ...  ..    ...    ...  ..    ...  ...   
291    31  41  75.6     358     120    21  26   80.8      8  11   72.7    0   
292    35  41  85.4     693     281    12  12  100.0     19  19  100.0    4   
293    39  47  83.0     735     239    13  16   81.3     22  24   91.7    4   
294    42  49  85.7     600     282    27  28  

In [22]:
# save links that have been downloaded already so that in the future one can only fetch data from new games
with open('../data/fbref_fixture_links.txt', 'w') as f:
    f.write('\n'.join(links))

In [23]:
old_fbref_data = pd.read_csv('../data/fbref_data.csv', index_col=0, header=[0,1])
#old_fbref_data.to_csv(f'../data/fbref_data_backup_week{gameweek}.csv')

In [24]:
fbref_data = pd.concat([old_fbref_data, fbref_data], axis=0)
print(f'Number of rows in fbref_data: {fbref_data.shape[0]}.')

Number of rows in fbref_data: 7171.


In [25]:
fbref_data.to_csv(f'../data/fbref_data.csv')

## 22-23 Match results

In [26]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
data = pd.read_html(url)
results = data[0].dropna(axis=0, how='all').copy()
results

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,1.2,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,Match Report,NaN
1,1.0,Sat,2022-08-06,12:30,Fulham,1.2,2–2,1.2,Liverpool,22207.0,Craven Cottage,Andy Madley,Match Report,NaN
2,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,Match Report,NaN
3,1.0,Sat,2022-08-06,15:00,Newcastle Utd,1.7,2–0,0.3,Nott'ham Forest,52245.0,St James' Park,Simon Hooper,Match Report,NaN
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.8,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,Match Report,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,38.0,Sun,2023-05-28,15:00,Everton,NaN,NaN,NaN,Bournemouth,NaN,Goodison Park,NaN,Head-to-Head,NaN
423,38.0,Sun,2023-05-28,15:00,Leicester City,NaN,NaN,NaN,West Ham,NaN,King Power Stadium,NaN,Head-to-Head,NaN
424,38.0,Sun,2023-05-28,15:00,Aston Villa,NaN,NaN,NaN,Brighton,NaN,Villa Park,NaN,Head-to-Head,NaN
425,38.0,Sun,2023-05-28,15:00,Leeds United,NaN,NaN,NaN,Tottenham,NaN,Elland Road,NaN,Head-to-Head,NaN


In [27]:
# fix Nott Forest team name
my_dict = {"Nott'ham Forest":'Nottingham Forest'}
results['Home'] = results['Home'].map(my_dict).fillna(results['Home'])
results['Away'] = results['Away'].map(my_dict).fillna(results['Away'])

In [28]:
results.to_csv('../data/PL_results_season22_23.csv')